In [126]:
################## Milestone2 ##################

In [127]:
### Download and unzip url
import numpy as np
from zipfile import ZipFile  
from urllib.request import urlretrieve
import os
import pandas as pd
from os import mkdir
### specifiying url of zip file
url = "https://www.dropbox.com/s/yr9irxy4ew1xqj9/5109581.zip?dl=1"
csvname = url [-15:-9]
# inputFile = "./DownloadedFile.zip"
# outputDir = "./DownloadedFile"

INPUT_FILE_SAVE_DIR = "./DownloadedFile.zip"
OUTPUT_DIR = "./DownloadFile/"
### put zip file in local directory -- this takes a while.. file is big
# urlretrieve(url, inputFile)
urlretrieve(url, INPUT_FILE_SAVE_DIR)
### unzip contents into output folder -- this takes a while.. file is big
# with ZipFile(inputFile) as zipObj:  
#     zipObj.extractall(outputDir)
with ZipFile(INPUT_FILE_SAVE_DIR) as zipObj:  
    zipObj.extractall(OUTPUT_DIR)
    

### Extract pic names in the folder
import os

file_list = os.listdir("./DownloadFile")
file_list.sort()

# remove .jpg
new_list = [s.replace(".jpg", "") for s in file_list]
# print(new_list)

### input csv 
import pandas as pd
CustomerList = pd.read_csv("./liveCustomerList.csv") #read the csv file (put 'r' before the path string to address any special characters in the path, such as '\'). Don't forget to put the file name at the end of the path + ".csv"
CustomerList['firstName'] = CustomerList['firstName'].str.upper()
CustomerList['lastName'] = CustomerList['lastName'].str.upper()
FraudList = pd.read_csv("./liveFraudList.csv")
FraudList["fraudster"] = 'NA'

#change custID list to Dataframe
#***seperate custID and loginAcct
jpg_custID = [s[0:4] for s in file_list]
jpg_loginAcct = [s[-10:-4] for s in file_list]

# print(jpg_custID)
# print(jpg_loginAcct)

custID = pd.DataFrame(jpg_custID, columns = ['custID'])
loginAcct = pd.DataFrame(jpg_loginAcct, columns = ['loginAcct'])

# change datatype
custID['custID']=custID['custID'].astype(int)
loginAcct['loginAcct']=loginAcct['loginAcct'].astype(int)
# print (custID.dtypes)
# print(custID)

# left join table custID and customerlist
custIDname = pd.merge(custID, CustomerList, on='custID', how='left')
custIDnameFraud = pd.merge(custIDname, FraudList, on=[
        'firstName', 'lastName'], how='left')
custIDnameFraud['fraudster'] = custIDnameFraud['fraudster'].fillna(0)

# FraudTestOnput.csv
output_fraud = custIDnameFraud[['custID', 'fraudster']]
print(output_fraud)

    custID fraudster
0     1129         0
1     1225         0
2     1316        NA
3     1354         0
4     1382         0
..     ...       ...
15    1762         0
16    1796         0
17    1803         0
18    1808         0
19    1820         0

[20 rows x 2 columns]


In [128]:
################## Milestone3 ##################
#print(custID)
 # previous downloaded file format
AcctInput = custID.copy()
AcctInput['loginAcct'] = jpg_loginAcct

    # left join input table and customerlist
custIDname_verify = pd.merge(
    AcctInput, CustomerList, on='custID', how='left')

    # input csv
BankAcct = pd.read_csv("./liveBankAcct.csv")
BankAcct['firstName'] = BankAcct['firstName'].str.upper()
BankAcct['lastName'] = BankAcct['lastName'].str.upper()
custIDname_verify = custIDname_verify.astype({"loginAcct": int})

    # left join with bankacct
AcctName_verify = pd.merge(custIDname_verify, BankAcct,
                               left_on='loginAcct', right_on='bankAcctID', how='left')

    # add conditional rightAcctFlag column
def f(AcctName_verify):
    if AcctName_verify['firstName_x'] == AcctName_verify['firstName_y'] and AcctName_verify['lastName_x'] == AcctName_verify['lastName_y']:
        val = 0
    else:
        val = 'NA'
    return val
AcctName_verify['rightAcctFlag'] = AcctName_verify.apply(f, axis=1)

    # FraudTestOnput.csv
output_verify = AcctName_verify[['custID', 'rightAcctFlag']]

In [129]:
#previous downloaded file format
AcctInput = custID.copy()
AcctInput['loginAcct'] = jpg_loginAcct
print(AcctInput)

    custID loginAcct
0     1129    439767
1     1225    428807
2     1316    616401
3     1354    575618
4     1382    385039
..     ...       ...
15    1762    770781
16    1796    771040
17    1803    323696
18    1808    786402
19    1820    356999

[20 rows x 2 columns]


In [130]:
### left join input table and customerlist
custIDname_verify = pd.merge(AcctInput, CustomerList, on='custID', how='left')

# b=BankAcct.dtypes
# print(b)
# input csv
BankAcct = pd.read_csv("./liveBankAcct.csv")
custIDname_verify=custIDname_verify.astype({"loginAcct":int})
# a=custIDname_verify.dtypes
# print(a)
### left join with bankacct
AcctName_verify = pd.merge(custIDname_verify, BankAcct, left_on='loginAcct', right_on='bankAcctID', how='left')


###add conditional rightAcctFlag column
def f(AcctName_verify):
    if AcctName_verify['firstName_x']==AcctName_verify['firstName_y'] and AcctName_verify['lastName_x']==AcctName_verify['lastName_y']:
        val = 0
    else:
        val = 1
    return val
AcctName_verify['rightAcctFlag'] = AcctName_verify.apply(f,axis=1)
print(AcctName_verify)

#FraudTestOnput.csv
output_verify = AcctName_verify[['custID','rightAcctFlag']]
print(output_verify)

    custID  loginAcct firstName_x  lastName_x  bankAcctID firstName_y  \
0     1129     439767     CECILIA     BOLOCCO      439767     Cecilia   
1     1225     428807       ERIKA      HAROLD      428807       Erika   
2     1316     616401        JACK       STRAW      616401        Jack   
3     1354     575618      JEREMY  GREENSTOCK      575618      Jeremy   
4     1382     385039        JOHN       MAYER      385039        John   
..     ...        ...         ...         ...         ...         ...   
15    1762     770781         TOM     DASCHLE      770781      Nelson   
16    1796     771040    VLADIMIR       PUTIN      771040    Vladimir   
17    1803     323696       WAYNE    FERREIRA      323696       Wayne   
18    1808     786402     WILLIAM   DONALDSON      786402     William   
19    1820     356999      YASSER      ARAFAT      356999      Sophia   

    lastName_y  rightAcctFlag  
0      Bolocco              1  
1       Harold              1  
2        Straw             

In [131]:
################## Milestone5 ##################

In [132]:
#read startbalance & bankTransactions 
acctStBalan = pd.read_csv('./Milestone5Files/startBalance.csv')
bankTrans = pd.read_csv('./Milestone5Files/bankTransactions.csv')

In [133]:
# left join loginAcct & startBalanceInfo
acctStBalan = loginAcct.merge(acctStBalan, left_on = 'loginAcct', right_on='bankAcctID', how = 'left')
# left join loginAcct & bankTransactions, append acctIDTrans to acctStBalanInfo
acctIDTrans = loginAcct.merge(bankTrans, left_on = 'loginAcct', right_on='bankAcctID', how = 'left')

acctIDTrans = acctIDTrans.rename({'transAmount': 'amt'}, axis =1)

#append acctIDTrans to acctStBalanInfo
acctsBalanInfo = acctStBalan.append(acctIDTrans, ignore_index = True)
acctsBalanInfo = acctsBalanInfo[acctsBalanInfo.amt > 200]

In [134]:
second_max_date = acctsBalanInfo.sort_values(["bankAcctID","date"], ascending=[1,0]).groupby("bankAcctID").agg({"date":lambda x: x.shift(-1).values[0]}).reset_index()
second_max_date.columns = second_max_date.columns.str.replace('date', 'second_max_date')
max_date = acctsBalanInfo.groupby('bankAcctID')['date'].max().reset_index()
max_date.columns = max_date.columns.str.replace('date', 'max_date')
# left join second_max_date and max_date
first_sec_max = max_date.merge(second_max_date, on = 'bankAcctID', how = 'left')
print(first_sec_max)

    bankAcctID    max_date second_max_date
0       282849  2020-02-10      2019-12-23
1       288395  2020-04-29      2020-04-27
2       315403  2020-04-20      2020-04-14
3       323696  2020-04-27      2020-04-13
4       356999  2020-04-22      2020-04-09
..         ...         ...             ...
14      616401  2020-04-24      2020-04-10
15      754976  2020-04-22      2020-04-08
16      770781  2020-04-20      2020-04-06
17      771040  2020-04-30      2020-04-15
18      786402  2020-04-30      2020-04-15

[19 rows x 3 columns]


In [135]:
pay_method = [
    ['2020-04-15','2020-04-30','2020-05-15'], 
    ['2020-04-17','2020-05-17','2020-06-17'],
    ['2020-04-20','2020-05-20','2020-06-20'],
    ['2020-04-10','2020-04-24','2020-05-08'],
    ['2020-04-17','2020-04-24','2020-05-01'],
    ['2020-04-30','2020-05-30','2020-06-30'],
    ['2020-04-13','2020-04-27', '2020-05-11'],
    ['2020-04-15','2020-05-15','2020-06-15'],
    ['2020-04-20','2020-04-27', '2020-05-04'],
    ['2020-04-23','2020-04-30', '2020-05-07'],
    ['2020-04-23','2020-05-23','2020-06-23'],
    ['2020-04-22','2020-05-22','2020-06-22'],
    ['2020-04-21', '2020-05-21','2020-06-21'],
    ['2020-04-10','2020-05-10','2020-06-10'],
    ['2020-04-15', '2020-04-29', '2020-05-13'],
    ['2020-04-13', '2020-05-13','2020-06-13'],
    ['2020-04-14', '2020-04-28', '2020-05-12'],
    ['2020-04-21','2020-04-28', '2020-05-05'], 
    ['2020-04-22', '2020-04-29', '2020-05-06'], 
    ['2020-04-16', '2020-04-30', '2020-05-14'], 
    ['2020-04-20', '2020-04-30', '2020-05-10'], 
    ['2020-04-27','2020-05-27','2020-06-27'],
    ['2020-04-16','2020-05-16','2020-06-16'],
    ['2020-04-27', '2020-04-30', '2020-05-03'],
    ['2020-04-10', '2020-04-17', '2020-04-24'], 
    #add
    ['2020-04-14', '2020-04-27', '2020-04-24'], 
    ['2020-04-03', '2020-04-17', '2020-05-01'], 
    ['2020-03-02', '2020-04-01', '2020-05-01'], #not sure
    ['2020-04-07', '2020-04-21', '2020-05-05'],
    ['2020-03-23', '2020-04-13', '2020-05-04'],
    ['2020-04-09', '2020-04-23', '2020-05-07'],
    ['2020-03-31', '2020-04-15', '2020-04-30'], #not sure
    ['2020-03-31', '2020-04-30', '2020-05-29'],#not sure
    ['2020-04-06', '2020-04-20', '2020-05-04'],
    ['2020-04-08', '2020-04-22', '2020-05-06'],
    ['2020-03-06', '2020-04-06', '2020-05-06'],
    ['2020-03-30', '2020-04-13', '2020-04-27'],
    ['2020-04-01', '2020-04-08', '2020-04-15'],
    ['2020-03-20', '2020-04-20', '2020-05-20'],
    ['2020-04-08', '2020-04-15', '2020-04-22'],
    
    
    ]
pay_pattern=pd.DataFrame(pay_method, columns = ['second_max_date','max_date','date'])
# print(pay_pattern)
# print(type(pay_pattern))

In [136]:
#Join first_sec_max and pay_pattern
pred_result = first_sec_max.merge(pay_pattern, on = ['max_date','second_max_date'], how = 'left')
#pd.set_option('display.max_rows', None)
pd.options.display.max_rows = 10
pred_result['date']=pred_result['date'].fillna('NA')
print(pred_result)
#count no-na
print(20-pred_result['date'].isna().sum())

    bankAcctID    max_date second_max_date        date
0       282849  2020-02-10      2019-12-23          NA
1       288395  2020-04-29      2020-04-27          NA
2       315403  2020-04-20      2020-04-14          NA
3       323696  2020-04-27      2020-04-13  2020-05-11
4       356999  2020-04-22      2020-04-09          NA
..         ...         ...             ...         ...
14      616401  2020-04-24      2020-04-10  2020-05-08
15      754976  2020-04-22      2020-04-08  2020-05-06
16      770781  2020-04-20      2020-04-06  2020-05-04
17      771040  2020-04-30      2020-04-15  2020-05-15
18      786402  2020-04-30      2020-04-15  2020-05-15

[19 rows x 4 columns]
20


In [137]:
pred_result_bankAcctID=pred_result[['bankAcctID','date']]
# left join loginAcct & startBalanceInfo
AcctInput=AcctInput.astype({"loginAcct":int})
pred_result_custID = pred_result_bankAcctID.merge(AcctInput, left_on = 'bankAcctID', right_on='loginAcct', how = 'left')
output_date=pred_result_custID[['custID','date']]
output_date.columns = ['custID','date1']
print(output_date)

    custID       date1
0     1709          NA
1     1700          NA
2     1703          NA
3     1803  2020-05-11
4     1820          NA
..     ...         ...
14    1316  2020-05-08
15    1526  2020-05-06
16    1762  2020-05-04
17    1796  2020-05-15
18    1808  2020-05-15

[19 rows x 2 columns]


In [138]:
################## Milestone1 ##################

In [139]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
# To install this module, run:
# python -m pip install Pillow
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person, QualityForRecognition

In [140]:
from data_processor import DataProcessor
import pandas as pd
data_processer = DataProcessor("dd")
data_processer.execute()
people = data_processer.persons
data = [[v.get_customer_id(), v.get_account_id(), v.fraud]
        for v in people.values()]
    #df = pd.DataFrame(data, columns = ['CustomerId', 'AccountId', 'verifiedId'])
df = pd.DataFrame(data, columns=['custID', 'AccountId', 'verifiedId'])
df['face'] = np.where((df['verifiedId'] == 0), 'NA', 0)
output_face = df[['custID', 'face']]
output_face['custID'] = output_face['custID'].astype(int)
print(output_face)

IndexError: list index out of range

In [ ]:
"""from data_processor import DataProcessor
import pandas as pd
data_processer = DataProcessor("dd")
data_processer.execute()"""

IndexError: list index out of range

In [ ]:
"""people = data_processer.persons
data = [[v.get_customer_id(), v.get_account_id(), v.fraud] for v in people.values()]
#df = pd.DataFrame(data, columns = ['CustomerId', 'AccountId', 'verifiedId'])
df = pd.DataFrame(data, columns = ['custID', 'AccountId', 'verifiedId'])
df['face'] = np.where((df['verifiedId']==0), 'NA',0)
output_face = df[['custID', 'face']]
output_face['custID']=output_face['custID'].astype(int)

# print(df)
print(output_face)"""

Empty DataFrame
Columns: [custID, face]
Index: []


In [ ]:
################### join output_fraud, output_verify, output_date, ***output_face(milestone1)*** together

In [ ]:
fraud_verify = output_fraud.merge(output_verify, on='custID',how = 'left')
fraud_verify_date = fraud_verify.merge(output_date, on='custID',how = 'left')
fraud_verify_date_face = fraud_verify_date.merge(output_face, on='custID',how = 'left')
print(fraud_verify_date_face)

    custID fraudster  rightAcctFlag       date1 face
0     1109         0              0          NA    0
1     1110         0              0  2020-04-27    0
2     1150         0              0  2020-05-15    0
3     1195        NA              0  2020-05-15   NA
4     1238         0              0  2020-05-01   NA
..     ...       ...            ...         ...  ...
21    1784         0              0          NA    0
22    1798         0              0          NA    0
23    1805         0              0  2020-05-08    0
24    1817         0              0          NA    0
25    1827         0              0          NA    0

[26 rows x 5 columns]


In [ ]:
#fraud_verify_date_face
output_prep = fraud_verify_date_face.copy()
output_prep['date'] = output_prep['date1']
output_prep['date'] = np.where((output_prep['fraudster']=='NA') | (output_prep['rightAcctFlag']=='NA') | (output_prep['date1']=='NA') | (output_prep['face']=='NA'), 'NA',output_prep['date1'])
print(output_prep)

    custID fraudster  rightAcctFlag       date1 face        date
0     1109         0              0          NA    0          NA
1     1110         0              0  2020-04-27    0  2020-04-27
2     1150         0              0  2020-05-15    0  2020-05-15
3     1195        NA              0  2020-05-15   NA          NA
4     1238         0              0  2020-05-01   NA          NA
..     ...       ...            ...         ...  ...         ...
21    1784         0              0          NA    0          NA
22    1798         0              0          NA    0          NA
23    1805         0              0  2020-05-08    0  2020-05-08
24    1817         0              0          NA    0          NA
25    1827         0              0          NA    0          NA

[26 rows x 6 columns]


In [ ]:
################## output csv ##################
output = output_prep[['custID','date']]
output.columns = ['loginID','date']
print(output)

    loginID        date
0      1109          NA
1      1110  2020-04-27
2      1150  2020-05-15
3      1195          NA
4      1238          NA
..      ...         ...
21     1784          NA
22     1798          NA
23     1805  2020-05-08
24     1817          NA
25     1827          NA

[26 rows x 2 columns]


In [ ]:
#write ID & fraudster columns to csv
output.to_csv(csvname+'.csv', index = False)